In [1]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

engine = create_engine('sqlite+pysqlite:///app/db/data.db', echo=True)

In [2]:
from sqlalchemy import select

from app.models.item import Item
from app.models.stock import Stock
from app.models.cat import Cat
from app.dao.db_base import Base

session = Session(engine)

In [3]:
stmt = (
    select(Item, Stock, Cat)
    .join(Item.stock)
    .join(Item.cat)
    .where(Stock.ready == False)
)

In [4]:
print(stmt)

SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = false


In [5]:
result = session.execute(stmt).scalars().all()

2025-05-15 14:53:23,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-15 14:53:23,410 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = 0
2025-05-15 14:53:23,412 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ()


In [6]:
from app.schemas.item import ItemInStock
items_on_stock = [ItemInStock.model_validate(item) for item in result]

2025-05-15 14:53:23,454 INFO sqlalchemy.engine.Engine SELECT stocks.reference AS stocks_reference, stocks.date AS stocks_date, stocks.consignor AS stocks_consignor, stocks.ready AS stocks_ready, stocks.note AS stocks_note, stocks.id AS stocks_id 
FROM stocks 
WHERE stocks.id = ?
2025-05-15 14:53:23,455 INFO sqlalchemy.engine.Engine [generated in 0.00192s] (1,)
2025-05-15 14:53:23,458 INFO sqlalchemy.engine.Engine SELECT cats.name AS cats_name, cats.cat AS cats_cat, cats.width AS cats_width, cats.weight AS cats_weight, cats.note AS cats_note, cats.id AS cats_id 
FROM cats 
WHERE cats.id = ?
2025-05-15 14:53:23,458 INFO sqlalchemy.engine.Engine [generated in 0.00041s] (3,)
2025-05-15 14:53:23,460 INFO sqlalchemy.engine.Engine SELECT stocks.reference AS stocks_reference, stocks.date AS stocks_date, stocks.consignor AS stocks_consignor, stocks.ready AS stocks_ready, stocks.note AS stocks_note, stocks.id AS stocks_id 
FROM stocks 
WHERE stocks.id = ?
2025-05-15 14:53:23,462 INFO sqlalchemy.

In [7]:
items_on_stock

[ItemInStock(id=1, lot='00245', pallet='0034', roll='0101', note='Item note', stock=Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=2, lot='00245', pallet='0034', roll='0103', note='Item note', stock=Stock(reference='FEU036/25', date=datetime.datetime(2025, 2, 10, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=3, lot='00430', pallet='0055', roll='1022', note='Item note', stock=Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=2200, weight=156.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=4, lot='20003', pallet='0204', roll

In [8]:
stocks = [item.stock for item in items_on_stock]

In [9]:
for stock in stocks:
    print(stock.date.strftime('%d.%m.%Y'))

02.02.2025
10.02.2025
02.02.2025
14.02.2025
02.02.2025


In [10]:
temp = [item.stock.date.strftime('%d.%m.%Y') for item in items_on_stock]

In [11]:
temp

['02.02.2025', '10.02.2025', '02.02.2025', '14.02.2025', '02.02.2025']

In [12]:
import copy

def format_item_date(item):
    cloned = copy.deepcopy(item)
    cloned.stock.date = cloned.stock.date.strftime('%d.%m.%Y')
    return cloned

formatted_items = [format_item_date(item) for item in items_on_stock]

In [13]:
formatted_items

[ItemInStock(id=1, lot='00245', pallet='0034', roll='0101', note='Item note', stock=Stock(reference='FEU034/25', date='02.02.2025', consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=2, lot='00245', pallet='0034', roll='0103', note='Item note', stock=Stock(reference='FEU036/25', date='10.02.2025', consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=3, lot='00430', pallet='0055', roll='1022', note='Item note', stock=Stock(reference='FEU034/25', date='02.02.2025', consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=2200, weight=156.0, note='Сгенерированное примечание к категории')),
 ItemInStock(id=4, lot='20003', pallet='0204', roll='0126', note='item note', stock=Stock(reference='FEU040/25', date='14

In [14]:
from sqlalchemy.orm import joinedload

stmt = (
    select(Item)
    .options(joinedload(Item.stock), joinedload(Item.cat))
)

print(stmt)

SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks_1.reference, stocks_1.date, stocks_1.consignor, stocks_1.ready, stocks_1.note AS note_1, stocks_1.id AS id_1, cats_1.name, cats_1.cat, cats_1.width, cats_1.weight, cats_1.note AS note_2, cats_1.id AS id_2 
FROM items LEFT OUTER JOIN stocks AS stocks_1 ON stocks_1.id = items.stock_id LEFT OUTER JOIN cats AS cats_1 ON cats_1.id = items.cat_id


In [18]:
result = session.execute(stmt).scalars().all()

2025-05-15 15:01:04,185 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks_1.reference, stocks_1.date, stocks_1.consignor, stocks_1.ready, stocks_1.note AS note_1, stocks_1.id AS id_1, cats_1.name, cats_1.cat, cats_1.width, cats_1.weight, cats_1.note AS note_2, cats_1.id AS id_2 
FROM items LEFT OUTER JOIN stocks AS stocks_1 ON stocks_1.id = items.stock_id LEFT OUTER JOIN cats AS cats_1 ON cats_1.id = items.cat_id
2025-05-15 15:01:04,186 INFO sqlalchemy.engine.Engine [cached since 104.6s ago] ()


In [19]:
from app.schemas.item import ItemInStock
items_on_stock = [ItemInStock.model_validate(item) for item in result]

In [23]:
print(len(items_on_stock), items_on_stock)

5 [ItemInStock(id=1, lot='00245', pallet='0034', roll='0101', note='Item note', stock=Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')), ItemInStock(id=2, lot='00245', pallet='0034', roll='0103', note='Item note', stock=Stock(reference='FEU036/25', date=datetime.datetime(2025, 2, 10, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=1800, weight=123.0, note='Сгенерированное примечание к категории')), ItemInStock(id=3, lot='00430', pallet='0055', roll='1022', note='Item note', stock=Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), cat=Cat(name='Visual', cat='film', width=2200, weight=156.0, note='Сгенерированное примечание к категории')), ItemInStock(id=4, lot='20003', pallet='0204', roll=

In [21]:
print(items_on_stock[0].stock.date)

2025-02-02 00:00:00


In [22]:
stmt = (
    select(Item)
    .join(Stock)
    .join(Cat)
)

print(stmt)

SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id
